In [1]:
#import sys
#!{sys.executable} -m pip install folium
#!{sys.executable} -m pip install --upgrade setuptools
#!{sys.executable} -m pip install ez_setup
#!{sys.executable} -m pip install unroll

In [140]:
import os
import pandas as pd
import numpy as np
import folium
import json
from folium.features import DivIcon
print(folium.__version__)

0.7.0


In [141]:
#Leitura do arquivo json para ler o mapa
geo_region_json = os.path.join('data','Brasil', 'geojs-35-mun.json')
geo_json_data = json.load(open(geo_region_json))

## Coloreando as regiones do mapa

In [142]:
#Inicializando o meio da tela do mapa de BRASILIA
m = folium.Map([-15.794906, -47.883656], zoom_start=4)
folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': 'green' if 'w' in feature['properties']['name'].lower() else 'red',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)
m.save(os.path.join('data', 'GeoJSON_and_choropleth_1.html'))

## Lendo os dados do Data Frame para unir com o mapa

In [143]:
df_Dados=pd.read_csv("data/Dados.csv",sep=',')
df_Dados.tail()

,id_region,2018_01_01,2018_01_02,2018_01_03,2018_01_04,2018_01_05,2018_01_06,2018_01_07,2018_01_08,2018_01_09,2018_01_10,2018_01_11,2018_01_12
640,Votorantim,1,2,3,4,5,6,7,8,9,10,11,12
641,Votuporanga,1,2,3,4,5,6,7,8,9,10,11,12
642,Zacarias,1,2,3,4,5,6,7,8,9,10,11,12
643,Chavantes,1,2,3,4,5,6,7,8,9,10,11,12
644,Estiva Gerbi,1,2,3,4,5,6,7,8,9,10,11,12


## Calculado o Centroide de cada region de los poligonos

In [146]:
#Criando um novo dataframe para guardar os centroides com seus respectivos ids (Nome_Region)
df_Centroides=pd.DataFrame(columns=['id_region','cord_centroide'])
#Filtrando as cordenadas do arquivo json para calcular os centrides
for Poligonos in range(len(geo_json_data['features'])):
    #Filtrando o id_Region de cada poligono
    ide=geo_json_data['features'][Poligonos]['properties']['id']
    #Filtro en listas de todos os pontos de cada poligono
    Lista_Coord_Poly=geo_json_data['features'][Poligonos]['geometry']['coordinates'][0]
    #Almazenando os valores "X" de cada ponto
    x = [p[0] for p in Lista_Coord_Poly]
    #Almazenando os valores "Y" de cada ponto
    y = [p[1] for p in Lista_Coord_Poly]
    centroid = [np.round(sum(y) / len(Lista_Coord_Poly),decimals=6), np.round(sum(x) / len(Lista_Coord_Poly),decimals=6)]
    df_Centroides.set_value(Poligonos,'id_region',ide)
    df_Centroides.set_value(Poligonos,'cord_centroide',centroid)
df_Centroides.head()

C:\Users\emily\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead

C:\Users\emily\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead



,id_region,cord_centroide
0,Adamantina,"[-21.583839, -51.069608]"
1,Adolfo,"[-21.283119, -49.657473]"
2,Aguaí,"[-22.033233, -47.059454]"
3,Águas da Prata,"[-21.920955, -46.700187]"
4,Águas de Lindóia,"[-22.47691, -46.600142]"


In [147]:
df_Dados_com_Coordenadas = pd.merge(df_Dados, df_Centroides, how='inner', on=['id_region'])
df_Dados_com_Coordenadas.tail()

,id_region,2018_01_01,2018_01_02,2018_01_03,2018_01_04,2018_01_05,2018_01_06,2018_01_07,2018_01_08,2018_01_09,2018_01_10,2018_01_11,2018_01_12,cord_centroide
640,Votorantim,1,2,3,4,5,6,7,8,9,10,11,12,"[-23.577462, -47.412144]"
641,Votuporanga,1,2,3,4,5,6,7,8,9,10,11,12,"[-20.483022, -50.016041]"
642,Zacarias,1,2,3,4,5,6,7,8,9,10,11,12,"[-21.110821, -50.056437]"
643,Chavantes,1,2,3,4,5,6,7,8,9,10,11,12,"[-23.031241, -49.728742]"
644,Estiva Gerbi,1,2,3,4,5,6,7,8,9,10,11,12,"[-22.233445, -46.943029]"


In [148]:
#calculando o valor Limite Superior dos cores (valor maximo da matriz)
Val_Max=df_Dados_com_Coordenadas.iloc[:,1:-1].max().max()
#calculando o valor Limite Inferior dos cores (valor minimo da matriz)
Val_Min=df_Dados_com_Coordenadas.iloc[:,1:-1].min().min()
Tamanho_do_Data_frame=len(df_Dados_com_Coordenadas)
#Insertando uma Linha Auxiliar np data frame que contemha os maximos valores
if (len(df_Dados_com_Coordenadas[df_Dados_com_Coordenadas.id_region=='Limite_Maxima'])<1)&(len(df_Dados_com_Coordenadas[df_Dados_com_Coordenadas.id_region=='Limite_Minima'])<1):
    df_Dados_com_Coordenadas.set_value(Tamanho_do_Data_frame,'id_region',"Limite_Minima")
    df_Dados_com_Coordenadas.set_value(Tamanho_do_Data_frame+1,'id_region',"Limite_Maxima")
    for fill_max in df_Dados_com_Coordenadas.iloc[:,1:-1].columns:
        df_Dados_com_Coordenadas.set_value(Tamanho_do_Data_frame,fill_max,Val_Min)
        df_Dados_com_Coordenadas.set_value(Tamanho_do_Data_frame+1,fill_max,Val_Max)
df_Dados_com_Coordenadas.tail()

C:\Users\emily\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead

C:\Users\emily\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead

C:\Users\emily\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead

C:\Users\emily\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead



,id_region,2018_01_01,2018_01_02,2018_01_03,2018_01_04,2018_01_05,2018_01_06,2018_01_07,2018_01_08,2018_01_09,2018_01_10,2018_01_11,2018_01_12,cord_centroide
642,Zacarias,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,"[-21.110821, -50.056437]"
643,Chavantes,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,"[-23.031241, -49.728742]"
644,Estiva Gerbi,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,"[-22.233445, -46.943029]"
645,Limite_Minima,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
646,Limite_Maxima,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,NaN


In [8]:
##Funcao que converte de um string a um HTML
#def entitify(text):
#    return ''.join('&#%d;' % ord(c) for c in text)

### Plotando com etiquetas de texto(formato=html) no mapa

In [151]:
def Pondo_Labels_poligonos(data,folium,Nome_da_Grafico,marker):
    Coordenada=data.cord_centroide.values.tolist()
    labels=data[Nome_da_Grafico].values.tolist()
    for point in range(len(Coordenada)):
        #Convertendo o string em um formato de html
        str_html=str(labels[point])
        #Pondo o label de texto em cada cebtroide da coordenada
        folium.map.Marker(
            Coordenada[point],
            icon=DivIcon(
                icon_size=(9,6),
                icon_anchor=(0,0),
                html='<div style="font-size: 6pt"><font color="red">%s</font></div>' % str_html
                #html=entitify(str_html),
                ),
            tooltip=None,
        ).add_to(m)
    Marker_Completado=folium.LayerControl().add_to(marker)
    #Guardando o mapa
    return Marker_Completado

### Plotando com estilos de Iconos no mapa

In [150]:
def Pondo_Labels_poligonos(data,folium,Nome_da_Grafico,marker):
    Coordenada=data.cord_centroide.values.tolist()
    labels=data[Nome_da_Grafico].values.astype(str).tolist()
    for point in range(len(Coordenada)):
        popup = folium.Popup(labels[point], parse_html=True)
        #Para seleccionar o tipo de Icone e modificar configuracao entrar aqui:
        #https://github.com/lvoogdt/Leaflet.awesome-markers
        folium.Marker(Coordenada[point], popup=popup, icon=folium.Icon(color='darkblue', icon_color='white', icon='coffee', angle=0, prefix='fa')).add_to(marker)
    Marker_Completado=folium.LayerControl().add_to(marker)
    #Guardando o mapa
    return Marker_Completado

In [339]:
#Dibujando conjunto de mapas de acordo com o tempo, assim como tambem coloreando de acordo a la quantidade de valores que tiene por o tempo
#percorrendo por o tempo
for Tempo in df_Dados_com_Coordenadas.columns[1:-1]:
    print (Tempo)
    #nome da data da grafica
    Nome_da_Grafico=Tempo
    #inicializando el mapa en el ponto de cordenada de Bradesco
    m = folium.Map([-23.549435, -46.774807], zoom_start=7)
    #Coloreando por intensidade cada poligono del mapa
    folium.Choropleth(
        geo_data=geo_json_data,
        name='choropleth',
        data=df_Dados_com_Coordenadas,
        columns=['id_region', Nome_da_Grafico],
        key_on='properties.id',
        fill_color='YlGn',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Intensidade de Cores Rate (%)'
    ).add_to(m)
    #Tirando a linha com os valores maximos e minimos
    df_Dados_com_Coordenadas_sem_Ultima_Linha=df_Dados_com_Coordenadas.iloc[:-2]
    #Insertando as etiquetas a cada poligono 
    Marker_Final=Pondo_Labels_poligonos(df_Dados_com_Coordenadas_sem_Ultima_Linha,folium,Nome_da_Grafico,m)
    Marker_Final.save(os.path.join('data', Nome_da_Grafico+'.html'))

2018_01_01
2018_01_02
2018_01_03
2018_01_04
2018_01_05
2018_01_06
2018_01_07
2018_01_08
2018_01_09
2018_01_10
2018_01_11
2018_01_12


### Capturando Html em Imagem (jpg ou png) _ _ 1

In [ ]:
#conda install selenium phantomjs pillow

In [370]:
from selenium import webdriver
def Criando_Imagem(Nome_Arquivo):
    driver = webdriver.Chrome(executable_path='C:\\Users\\emily\\Documents\\Juan\\WebChrome\\chromedriver_win32\\chromedriver.exe')
    driver.get('C:/Users/emily/Documents/Juan/data/'+Nome_Arquivo+'.html')
    driver.save_screenshot('data/'+Nome_Arquivo+'.png')
    driver.quit()

In [406]:
Lista_de_Arquivos=['2018_01_02']
for Arquivo in df_Dados_com_Coordenadas.columns[1:-1]:
    Criando_Imagem(Arquivo)
    print ('Imagem '+Arquivo+'.png Guardada')

Imagem 2018_01_01.png Guardada
Imagem 2018_01_02.png Guardada
Imagem 2018_01_03.png Guardada
Imagem 2018_01_04.png Guardada
Imagem 2018_01_05.png Guardada
Imagem 2018_01_06.png Guardada
Imagem 2018_01_07.png Guardada
Imagem 2018_01_08.png Guardada
Imagem 2018_01_09.png Guardada
Imagem 2018_01_10.png Guardada
Imagem 2018_01_11.png Guardada
Imagem 2018_01_12.png Guardada


### Capturando Html em Imagem (jpg ou png) _ _ 2

In [ ]:
#conda install -c auto pdfcrowd
import sys
!{sys.executable} -m pip install imgkit

In [344]:
Name_Html="cara"
File_html=r'C:/Users/emily/Documents/Juan/data/'+Name_Html+'.html'
options = {'width': 1400, 'disable-smart-width': '','height' : 1200,'quality': 94}
with open(File_html) as f:
    imgkit.from_file(f, 'C:/Users/emily/Documents/Juan/data/out.jpg',config=config,options=options)
    print ('Imagem "'+Name_Html +'" Guardada!')

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Imagem "cara" Guardada!


In [273]:
output_file='C:/Users/emily/Documents/Juan/data/out.jpg'
html_string='C:/Users/emily/Documents/Juan/data/2018_01_02.html'
config = imgkit.config(wkhtmltoimage='C:/Program Files/wkhtmltopdf/bin/wkhtmltoimage.exe')
imgkit.from_string(html_string, output_file, config=config)

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


True

### Converter imagens em Videio

In [230]:
#INSTALAR ESTES PAQUETES
#conda install -c conda-forge opencv
#conda install -c conda-forge/label/gcc7 opencv 
#conda install -c conda-forge/label/broken opencv
#conda install -c conda-forge/label/cf201901 opencv

In [415]:
import cv2
import numpy as np
import os
Imagenes=['2018_01_01','2018_01_02','2018_01_03','2018_01_04','2018_01_05']

# Playing video from file:
cap = cv2.VideoCapture(r'C:/Users/emily/Documents/Juan/data/PELICULA.mp4')

try:
    if not os.path.exists('data'):
        os.makedirs('data')
        print ("Náo existiu")
except OSError:
    print ('Error: Creating directory of data')

currentFrame = 0
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    print (ret)
    print (frame)
    # Saves image of the current frame in jpg file
    name = 'C:/Users/emily/Documents/Juan/data/' + str(Imagenes[currentFrame]) + '.png'
    print (name)
    print ('Creating...' + name)
    cv2.imwrite(name, frame)

    # To stop duplicate images
    currentFrame += 1
    if currentFrame >4:
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

False
None
C:/Users/emily/Documents/Juan/data/2018_01_01.png
Creating...C:/Users/emily/Documents/Juan/data/2018_01_01.png
False
None
C:/Users/emily/Documents/Juan/data/2018_01_02.png
Creating...C:/Users/emily/Documents/Juan/data/2018_01_02.png
False
None
C:/Users/emily/Documents/Juan/data/2018_01_03.png
Creating...C:/Users/emily/Documents/Juan/data/2018_01_03.png
False
None
C:/Users/emily/Documents/Juan/data/2018_01_04.png
Creating...C:/Users/emily/Documents/Juan/data/2018_01_04.png
False
None
C:/Users/emily/Documents/Juan/data/2018_01_05.png
Creating...C:/Users/emily/Documents/Juan/data/2018_01_05.png


In [391]:
def make_video(images, outimg=None, fps=5, size=None,
               is_color=True, format="XVID"):
    """
    Create a video from a list of images.
 
    @param      outvid      output video
    @param      images      list of images to use in the video
    @param      fps         frame per second
    @param      size        size of each frame
    @param      is_color    color
    @param      format      see http://www.fourcc.org/codecs.php
    @return                 see http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
 
    The function relies on http://opencv-python-tutroals.readthedocs.org/en/latest/.
    By default, the video will have the size of the first image.
    It will resize every image to this size before adding them to the video.
    """
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    print(fourcc)
    outvid='C:/Users/emily/Documents/Juan/data/'
    vid = None
    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
        print (vid.write(img))
    vid.release()
    return vid

In [234]:
from IPython.core.display import HTML
HTML('<a href="https://www.wikipedia.org/">link</a>')

In [235]:
%%html
<a href="https://www.google.com/">link</a>

In [ ]:
#http://www.pmf.sc.gov.br/arquivos/arquivos/pdf/04_01_2010_9.58.44.08ae0276e7919fbaf1a4bbd72c7d06df.PDF
#http://www.pmf.sc.gov.br/arquivos/arquivos/pdf/04_01_2010_10.27.25.2b615e6755138defe1bdb00f1c86031f.PDF
#https://github.com/caspg/datamaps.co/tree/master/public/data/topo
#https://github.com/tbrugz/geodata-br
#https://github.com/fititnt/gis-dataset-brasil/blob/master/microrregiao/topojson/microrregiao.json